## Deniosing Data with FFT

- Reference: https://www.kaggle.com/code/faressayah/signal-processing-with-python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
# Create synthetic signal
dt = 0.001
t = np.arange(0, 1, dt)
signal = np.sin(2 * np.pi * 50 * t) + np.sin(2 * np.pi * 120 * t) # Sum of 2 Sequencies
signal_clean = signal
signal = signal + 2.5 * np.random.randn(len(t)) # Add some noise
min_signal, max_signal = signal.min(), signal.max()

In [ ]:
plt.plot(t, signal, color='c', linewidth=1.5, label='Noisy')
plt.plot(t, signal_clean, color='k', linewidth=2, label='Clean')
plt.xlim(t[0], t[-1])
plt.xlabel('t axis')
plt.ylabel('Vals')
plt.legend()

In [ ]:
# Compute the Fast Fourier Transform (FFT)

n = len(t)
fhat = np.fft.fft(signal, n)                 # Compute the FFT
psd = fhat * np.conj(fhat) / n
freq = (1 / (dt * n)) * np.arange(n)    # frequency array
idxs_half = np.arange(1, np.floor(n / 2), dtype=np.int32)  # first half index

In [ ]:
fig, axs = plt.subplots(2, 1)

plt.sca(axs[0])
plt.plot(t, signal, color='c', linewidth=1.5, label='Noisy')
plt.plot(t, signal_clean, color='k', linewidth=2, label='Clean')
plt.xlim(t[0], t[-1])
plt.xlabel('t axis')
plt.ylabel('Vals')
plt.legend()

plt.sca(axs[1])
plt.plot(freq[idxs_half], psd[idxs_half], color='c', linewidth=2, label='PSD Noisy')
plt.xlim(freq[idxs_half[0]], freq[idxs_half[-1]])
plt.xlabel('t axis')
plt.ylabel('Vals')
plt.legend()

plt.tight_layout()

In [ ]:
threshold = 100
psd_idxs = psd > threshold # array of 0 and 1
psd_clean = psd * psd_idxs # zero out all the unnecessary powers
fhat_clean = psd_idxs * fhat # used to retreive the signal

signal_filtered = np.fft.ifft(fhat_clean) # inverse fourier transform

In [ ]:
# plt.rcParams['figure.figsize'] = [8,10]
fig, axs = plt.subplots(4, 1)

plt.sca(axs[0])
plt.plot(t, signal, color='b', linewidth=0.5, label='Noisy')
plt.plot(t, signal_clean, color='r', linewidth=1, label='Clean')
plt.ylim(min_signal, max_signal)
plt.xlabel('t axis')
plt.ylabel('Vals')
plt.legend()

plt.sca(axs[1])
plt.plot(freq[idxs_half], np.abs(psd[idxs_half]), color='b', linewidth=0.5, label='PSD noisy')
plt.xlabel('Frequencies in Hz')
plt.ylabel('Amplitude')
plt.legend()

plt.sca(axs[2])
plt.plot(freq[idxs_half], np.abs(psd_clean[idxs_half]), color='r', linewidth=1, label='PSD clean')
plt.xlabel('Frequencies in Hz')
plt.ylabel('Amplitude')
plt.legend()

plt.sca(axs[3])
plt.plot(t, signal_filtered, color='r', linewidth=1, label='Clean Signal Retrieved')
plt.xlim(t[0], t[-1])
plt.ylim(min_signal, max_signal)
plt.xlabel('t axis')
plt.ylabel('Vals')
plt.legend()

plt.tight_layout()

## Spectrogram

In [ ]:
dt = 0.001
t = np.arange(0, 2, dt)
f0 = 50
f1 = 250
t1 = 2
x = np.cos(2 * np.pi * t * (f1 - f0) * np.power(t, 2) / (3 * t1 ** 2))

fs = 1 / dt

# plt.rcParams['figure.figsize'] = [8,6]
plt.figure(figsize=(8, 8))

plt.subplot(2, 2, 1)
plt.specgram(x, NFFT=128, Fs=1/dt, noverlap=120, cmap='jet_r')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.colorbar()

In [ ]:
plt.plot(t, x)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

## Compress image using FFT

In [ ]:
import cv2

In [ ]:
!wget https://radiologypics.files.wordpress.com/2013/01/left-sided-ivc-2.jpg sample.jpg

In [ ]:
img = cv2.imread("left-sided-ivc-2.jpg", 0)
plt.imshow(img, cmap="gray")
plt.xticks([]), plt.yticks([])  # remove tick marks
plt.show()

In [ ]:
f = np.fft.fft2(img)  #the image 'img' is passed to np.fft.fft2() to compute its 2D Discrete Fourier transform f
mag = 20*np.log(np.abs(f))
plt.imshow(mag, cmap = 'gray') #cmap='gray' parameter to indicate that the image should be displayed in grayscale.
plt.title('Magnitude Spectrum')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
fshift = np.fft.fftshift(f)
mag = 20*np.log(np.abs(fshift))
plt.imshow(mag, cmap = 'gray')
plt.title('Centered Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
import math

def calculate_distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def generate_low_pass(d0: int, im_size: tuple):
    """
    Generate 2D low-pass filter from a given image size

    d0: diameter of a generated a circular 0, 1
    """
    base = np.zeros(im_size[:2])
    rows, cols = im_size[:2]
    center = (rows / 2, cols / 2)
    for x in range(cols):
        for y in range(rows):
            if calculate_distance((y, x), center) < d0:
                base[y, x] = 1
    return base

In [ ]:
d0 = 50
plt.imshow(np.abs(generate_low_pass(d0, img.shape)), cmap='gray')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
plt.imshow(
    np.log(1 + np.abs(fshift * generate_low_pass(d0, img.shape))),
    cmap='gray'
)
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
f_filtered = fshift * generate_low_pass(d0, img.shape)
img_rec = np.fft.ifft2(np.fft.ifftshift(f_filtered))

plt.imshow(np.abs(img_rec), cmap='gray')
plt.xticks([]), plt.yticks([])
plt.show()